In [12]:
import pickle
import os

In [4]:
with open("sector_nse.pkl", "rb") as f:
    sector_data = pickle.load(f)

In [6]:
sector_map = {}
for ticker, sectors in sector_data.items():
    i = 0
    for sector in sectors[2:]:
        if i == 0:
            sector = "MES_"+sector
        elif i == 1:
            sector = "SES_"+sector
        elif i == 2:
            sector = "IND_"+sector
        elif i == 3:
            sector = "BIN_"+sector
        i += 1
        if sector not in sector_map:
            sector_map[sector] = []
        sector_map[sector].append(ticker)

print(sector_map)

In [8]:
sector_id = {}
counter = [1,100,10000,1000000]
for sector, tickers in sector_map.items():
    if sector.split("_")[0] == "MES":
        sector_id[sector] = counter[0]
        counter[0] += 1
    elif sector.split("_")[0] == "SES":
        sector_id[sector] = counter[1]
        counter[1] += 1
    elif sector.split("_")[0] == "IND":
        sector_id[sector] = counter[2]
        counter[2] += 1
    elif sector.split("_")[0] == "BIN":
        sector_id[sector] = counter[3]
        counter[3] += 1
print(sector_id)



{'MES_Financial Services': 1, 'SES_Financial Services': 100, 'IND_Finance': 10000, 'BIN_Other Financial Services': 1000000, 'MES_Diversified': 2, 'SES_Diversified': 101, 'IND_Diversified': 10001, 'BIN_Diversified': 1000001, 'MES_Industrials': 3, 'SES_Capital Goods': 102, 'IND_Electrical Equipment': 10002, 'BIN_Heavy Electrical Equipment': 1000002, 'MES_Commodities': 4, 'SES_Construction Materials': 103, 'IND_Cement & Cement Products': 10003, 'BIN_Cement & Cement Products': 1000003, 'IND_Industrial Products': 10004, 'BIN_Castings & Forgings': 1000004, 'BIN_Iron & Steel Products': 1000005, 'IND_Banks': 10005, 'BIN_Other Bank': 1000006, 'MES_Healthcare': 5, 'SES_Healthcare': 104, 'IND_Pharmaceuticals & Biotechnology': 10006, 'BIN_Pharmaceuticals': 1000007, 'SES_Chemicals': 105, 'IND_Chemicals & Petrochemicals': 10007, 'BIN_Specialty Chemicals': 1000008, 'BIN_Housing Finance Company': 1000009, 'SES_Metals & Mining': 106, 'IND_Metals & Minerals Trading': 10008, 'BIN_Trading - Minerals': 100

In [14]:
sector_hypergraph = open("sector_hypergraph_nifty500.txt", "w")
sector_hypergraph.write("sector\tdescription\tnode_list\n")

for sector, node_list in sector_map.items():
    node_list_str = "\t".join([str(x) for x in node_list])
    sector_hypergraph.write("{0}\t{1}\t{2}\n".format(sector_id[sector], sector, node_list_str))
sector_hypergraph.close()

In [24]:
INDEX = "nifty500" 

directory = "../../data/" + INDEX + "/"

ticker_list = {}
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        ticker, name = filename.split("-")
        name = name[:-5]
        ticker_list[ticker] = name

In [18]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

uri = "neo4j+s://ef378426.databases.neo4j.io"
user = "neo4j"
password = "_6AluugfMr__UTW5tfW2f_lmaqyIzvttqMut4wOA7D8"
app = App(uri, user, password)

In [17]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_relation(self, id1, id2, category, relation, name2, relation_id, name):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.execute_write(
                self._create_and_return_relation, id1, id2, category, relation, name2, relation_id, name)
            for row in result:
                print("Created node between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

    @staticmethod
    def _create_and_return_relation(tx, id1, id2, category, relation, name2, relation_id, name):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        
        query = (
            
            "MATCH (a:Asset) WHERE a.ticker = $ticker1 "
            "RETURN a"
        )
        result = tx.run(query, ticker1=id1, ticker2=id2)

        if len(list(result)) == 0:
            print("Creating asset 1 ", id1)
            query = (
            "CREATE (p1:Asset { ticker: $ticker1, name: $name}) "
            )
            result = tx.run(query, ticker1=id1, name=name)

        query = (
            "MATCH (c:"+category+") WHERE c.name = $name AND c.id = $id2 "
            "RETURN c"
        )
        result = tx.run(query, id2=id2, category=category, name=name2)
        
        if len(list(result)) == 0:
            query = (
                "CREATE (c1:"+category+" { name: $name, id: $id2 }) "
            )
            result = tx.run(query, name=name2, id2=id2)

        query = (
            "MATCH (a:Asset), (c:"+category+") WHERE a.ticker = $ticker1 AND c.id = $id2 "
            "CREATE (a)-[:"+relation+" { id: $relation_id}]->(c) "
            "RETURN a, c"
        )
        result = tx.run(query, ticker1=id1, id2=id2, relation_id=relation_id)
        try:
            return [{"p1": row["a"]["ticker"], "p2": row["c"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_person(self, person_name):
        with self.driver.session(database="neo4j") as session:
            result = session.execute_read(self._find_and_return_person, person_name)
            for row in result:
                print("Found person: {row}".format(row=row))

    @staticmethod
    def _find_and_return_person(tx, person_name):
        query = (
            "MATCH (p:Person) "
            "WHERE p.name = $person_name "
            "RETURN p.name AS name"
        )
        result = tx.run(query, person_name=person_name)
        return [row["name"] for row in result]

In [26]:
for sector, node_list in sector_map.items():
    if sector.split("_")[0] == "MES":
        relation = "MacroEconomic_Sector"
        relation_id = "C1"
    elif sector.split("_")[0] == "SES":
        relation = "Sector"
        relation_id = "C2"
    elif sector.split("_")[0] == "IND":
        relation = "Industry"
        relation_id = "C3"
    elif sector.split("_")[0] == "BIN":
        relation = "Basic_Industry"
        relation_id = "C4"
        
    for tickers in node_list:
        tickers = tickers.replace("-", "_")
        app.create_relation(tickers, sector_id[sector], "Classification", relation, sector, relation_id, ticker_list[tickers])

Creating asset 1  360ONE
Created node between: 360ONE, MES_Financial Services
Creating asset 1  AUBANK
Created node between: AUBANK, MES_Financial Services
Creating asset 1  AAVAS
Created node between: AAVAS, MES_Financial Services
Creating asset 1  ABCAPITAL
Created node between: ABCAPITAL, MES_Financial Services
Creating asset 1  ANGELONE
Created node between: ANGELONE, MES_Financial Services
Creating asset 1  APTUS
Created node between: APTUS, MES_Financial Services
Creating asset 1  AXISBANK
Created node between: AXISBANK, MES_Financial Services
Creating asset 1  BSE
Created node between: BSE, MES_Financial Services
Creating asset 1  BAJFINANCE
Created node between: BAJFINANCE, MES_Financial Services
Creating asset 1  BAJAJFINSV
Created node between: BAJAJFINSV, MES_Financial Services
Creating asset 1  BAJAJHLDNG
Created node between: BAJAJHLDNG, MES_Financial Services
Creating asset 1  BANDHANBNK
Created node between: BANDHANBNK, MES_Financial Services
Creating asset 1  BANKBARODA